#### Imports

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from skopt import BayesSearchCV
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from scikeras.wrappers import KerasRegressor
from keras.callbacks import EarlyStopping
import numpy as np
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras.metrics import MeanSquaredError
from keras.regularizers import l2
from keras.layers import BatchNormalization
from skopt.space import Categorical
from sklearn.metrics import mean_squared_error
import json

In [ ]:
final_dataset = pd.read_csv('../../Datasets/final_dataset.csv', sep='\t')

In [ ]:
final_dataset.head()

In [ ]:
columns = ['plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'close_quarter', 'cat1', 'cat2', 'cat3', 'close_quarter', 'y']
non_scalable_columns = ['cat1', 'cat2', 'cat3', 'plan_precios_cuidados', 'close_quarter']
n_features = len(columns)

In [ ]:
categorical_columns = ['cat1', 'cat2', 'cat3']

# Transformar las columnas categóricas en numéricas
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    final_dataset[col] = le.fit_transform(final_dataset[col])
    label_encoders[col] = le

In [ ]:
grouped = final_dataset.groupby(['product_id', 'cat1', 'cat2', 'cat3'])
display(grouped)

In [ ]:
scalers = {col: {} for col in columns}
scaled_data_list = []

for (product_id, _, _, _ ), group in grouped:
    scaled_group = group.copy()
    for col in columns:
        if col not in non_scalable_columns:
            scaler = StandardScaler()
            scaled_group[col] = scaler.fit_transform(group[[col]])
            scalers[col][product_id] = scaler
    scaled_data_list.append(scaled_group)

# Combinar todos los datos escalados en un solo DataFrame
scaled_data = pd.concat(scaled_data_list)

In [ ]:
display(scaled_data[columns])

In [ ]:
# scaled_data[scaled_data['cat1'] != 0]

#### Funcion para preparar los datos y crear el modelo

El objetivo es predecir 2 dias en el futuro, por lo que la idea es re-armar el dataset. Donde el valor de X sera el conjunto de datos hasta N-2 e Y va a ser N (siendo N la cantidad de ventas para ese producto)

In [ ]:
def prepare_data(data,validation=False):
    X, y = [], []
    unique_product_ids = data['product_id'].unique()
    
    max_seq_length = 0
    
    for product_id in unique_product_ids:
        product_data = data[data['product_id'] == product_id].copy()
        product_values = product_data[columns].values
        
        max_seq_length = max(max_seq_length, len(product_values))
    
    for product_id in unique_product_ids:
        product_data = data[data['product_id'] == product_id].copy()
        product_values = product_data[columns].values
        
        pad_length = max_seq_length - len(product_values)
        
        # Aplicar padding con ceros a la izquierda para igualar la longitud
        padded_product_values = np.pad(product_values[:, :-1], ((pad_length, 0), (0, 0)), mode='constant')
        
        if validation:
            X.append(padded_product_values)
            y.append(product_id)
        else:
            # Todas las observaciones menos las últimas dos. Como despues necesito agregar esos 2 registros
            # para predecir el future, le agrego un pad left extra de 2 para no romper el shape de la red
            X.append(np.pad(padded_product_values, ((2, 0), (0, 0)), mode='constant')[:-2])
            y.append(product_values[-1, -1])      # Última observación
    
    if validation:
        return np.array(X), np.array(y), max_seq_length
    else:
        return np.array(X), np.array(y), max_seq_length


Definimos la funcion para crear el modelo LSTM, sobre este se ejecutara la optimizacion bayesiana

In [ ]:
n_steps = 2  # número de pasos de tiempo
epochs = 100
batch_size = 32
predictions = []

In [ ]:
data = scaled_data[columns].values
X, y, max_seq_length = prepare_data(scaled_data)

#### Revisamos que la funcion genere registros coherentes

In [ ]:
# Muestro todos los registros del ultimo producto
unique_product_ids = scaled_data['product_id'].unique()
product_data = scaled_data[scaled_data['product_id'] == unique_product_ids[0]] #<= cambiar el indice entre 0 y -1 para ver la diferencia agregada por el padleft
product_values = product_data[columns].values
display(product_values)

In [ ]:
display(X[0]) #<= cambiar el indice entre 0 y -1 para ver la diferencia agregada por el padleft

In [ ]:
display(y[0]) #<= cambiar el indice entre 0 y -1

In [ ]:
display(len(X[0]))

#### Entrenamiento del modelo

In [ ]:
display(max_seq_length)

In [ ]:
from keras.regularizers import l2

def create_model(units, learning_rate, optimizer='adam', dropout=0.1, activation="tanh", depth=2, l2_penalty=1e-6):
    display(f"Creating model with parameters: units={units}, learning_rate={learning_rate}, "
                f"optimizer={optimizer}, dropout={dropout}, activation={activation}, "
                f"depth={depth}, l2_penalty={l2_penalty}")
    
    model = Sequential()
    
    for i in range(depth):
        if i == 0:
            model.add(LSTM(units=units, return_sequences=True, input_shape=(max_seq_length, n_features - 1), activation=activation, kernel_regularizer=l2(l2_penalty)))
            model.add(BatchNormalization())
        else:
            model.add(LSTM(units=units, return_sequences=True, activation=activation, kernel_regularizer=l2(l2_penalty)))
            model.add(Dropout(dropout))
            model.add(BatchNormalization())
    
    model.add(LSTM(units=units, return_sequences=False, activation=activation, kernel_regularizer=l2(l2_penalty)))
    model.add(Dropout(dropout))
    
    model.add(Dense(1))
    
    if optimizer == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    
    # Usamos el error cuadratico medio, porque penaliza errores grandes. Como tenemos ciertos
    # productos "estrella", para los cuales hay mas ventas, y tener un error en lo mismos
    # puede implicar una gran diferencia de las estimaciones, decidimos probar con esta funcion de perdida.
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[MeanSquaredError()])
    return model

#### Armado de los modelos

In [ ]:
param_space = {
    'depth': [1, 2, 3, 4],#, 5, 6],
    'activation': ['relu', 'tanh'], #'selu', 'swish'
    'units': [64, 128, 256, 512],
    'dropout': [0.1, 0.2, 0.3],
    'learning_rate': Categorical(categories=(0.0001, 0.001), prior=None),#0.01
    'epochs': [10, 20, 30, 50, 100],
    'batch_size': Categorical([32, 128], prior=None),#[32, 64, 128],
    'optimizer': ['adam', 'rmsprop', 'sgd'],
    'l2_penalty': [1e-6, 1e-2, 1e-1]
}

In [ ]:
# early_stopping = EarlyStopping(monitor='val_loss', patience=5) mean_squared_error
early_stopping = EarlyStopping(monitor='mean_squared_error', patience=5)
callbacks = [early_stopping]

model = KerasRegressor(
    build_fn=create_model,
    verbose=1,
    units=64,
    learning_rate=0.01,
    dropout=0.1,
    activation="tanh",
    depth=1,
    l2_penalty=1e-6,
    callbacks = callbacks
)

# Como ahora cada observacion, representa la serie de un producto, no hace falta usar 
# un cv especifico de time series, con el cv normal deberia ser suficiente.
# cv = TimeSeriesSplit(n_splits=5).split(X)

bayes_search = BayesSearchCV(
    model,
    param_space,
    # n_iter=50,
    cv=10,
    verbose=1,
    # n_jobs=-1,
    scoring='neg_mean_squared_error',
)

In [ ]:
bayes_search.fit(X, y, verbose=1)

#### Guardo los mejores hiperparametros

In [164]:
best_params = bayes_search.cv_results_['params'][bayes_search.best_index_]
display(best_params)

best_model = create_model(
    units=best_params['units'],
    learning_rate=best_params['learning_rate'],
    optimizer=best_params['optimizer'],
    dropout=best_params['dropout'],
    activation=best_params['activation'],
    depth=best_params['depth'],
    l2_penalty=best_params['l2_penalty'],
)
best_model.fit(X, y, epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=1)

display(best_params)
display(best_model)

OrderedDict([('activation', 'tanh'),
             ('batch_size', 32),
             ('depth', 1),
             ('dropout', 0.1),
             ('epochs', 100),
             ('l2_penalty', 1e-06),
             ('learning_rate', 0.001),
             ('optimizer', 'adam'),
             ('units', 64)])

'Creating model with parameters: units=64, learning_rate=0.001, optimizer=adam, dropout=0.1, activation=tanh, depth=1, l2_penalty=1e-06'

Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - loss: 0.8736 - mean_squared_error: 0.8735
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.5978 - mean_squared_error: 0.5977
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.6777 - mean_squared_error: 0.6776
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.5507 - mean_squared_error: 0.5506
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.5308 - mean_squared_error: 0.5307
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.4731 - mean_squared_error: 0.4730
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.5109 - mean_squared_error: 0.5107
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.4588 - mean_squared_error: 0.4587
Epoch 9/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.4781 - mean_squared_error: 0.4780
Epoch 10/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.4032 - mean_squared_error: 0.4031
Epoch 11/100
25/25 ━━━━━━━━━━━━━━━━━━━━

OrderedDict([('activation', 'tanh'),
             ('batch_size', 32),
             ('depth', 1),
             ('dropout', 0.1),
             ('epochs', 100),
             ('l2_penalty', 1e-06),
             ('learning_rate', 0.001),
             ('optimizer', 'adam'),
             ('units', 64)])

<Sequential name=sequential_1007, built=True>

In [165]:
y_pred = best_model.predict(X)
mean_squared_error(y, y_pred)

25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step


0.1753564664479838

In [166]:
# Obtener los mejores hiperparámetros
best_params = bayes_search.cv_results_['params'][bayes_search.best_index_]

# Guardar los mejores hiperparámetros en un archivo JSON
with open('best_params.json', 'w') as file:
    json.dump(best_params, file)

print('Los mejores hiperparámetros han sido guardados en best_params.json.')


Los mejores hiperparámetros han sido guardados en best_params.json.


In [167]:
with open('best_params.json', 'r') as file:
    best_params = json.load(file)

# Reconstruir el modelo con los mejores hiperparámetros
mejor_modelo = create_model(units=best_params['units'], learning_rate=best_params['learning_rate'], optimizer=best_params['optimizer'], dropout=best_params['dropout'], activation=best_params['activation'], depth=best_params['depth'], l2_penalty=best_params['l2_penalty'])
mejor_modelo.fit(X, y, epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=1)
# Verificar la configuración del nuevo modelo
print("Configuración del nuevo modelo:")
print(mejor_modelo.summary())

'Creating model with parameters: units=64, learning_rate=0.001, optimizer=adam, dropout=0.1, activation=tanh, depth=1, l2_penalty=1e-06'

Epoch 1/100


/Users/mmarchetta/Desktop/LaboratorioIII/my_env/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 34ms/step - loss: 0.8400 - mean_squared_error: 0.8398
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.5540 - mean_squared_error: 0.5539
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.6266 - mean_squared_error: 0.6264
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.5233 - mean_squared_error: 0.5232
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.5568 - mean_squared_error: 0.5567
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.5635 - mean_squared_error: 0.5634
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 0.5450 - mean_squared_error: 0.5449
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.5763 - mean_squared_error: 0.5762
Epoch 9/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.4953 - mean_squared_error: 0.4952
Epoch 10/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 0.4914 - mean_squared_error: 0.4913
Epoch 11/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/ste

Model: "sequential_1008"

...

 Total params: 155,845 (608.77 KB)

 Trainable params: 51,905 (202.75 KB)

 Non-trainable params: 128 (512.00 B)

 Optimizer params: 103,812 (405.52 KB)

None


#### Hago las predicciones

In [168]:
# best_params = BayesSearchCV.best_params_

# best_units = best_params['build_fn__units']
# best_dropout_rate = best_params['build_fn__dropout_rate']
# best_optimizer = best_params['build_fn__optimizer']

# best_model = create_model(units=best_units, dropout_rate=best_dropout_rate, optimizer=best_optimizer)
# best_model.fit(X, y)

In [169]:
X_f, product_ids, _ = prepare_data(scaled_data, validation=True)

In [170]:
scaled_predictions = mejor_modelo.predict(X_f)
display(scaled_predictions)

25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 63ms/step


array([[-1.56514525e-01],
       [ 3.78918052e-01],
       [-1.20398891e+00],
       [-1.03173518e+00],
       [-1.09153891e+00],
       [-1.16096473e+00],
       [-1.01697111e+00],
       [-1.66051745e+00],
       [ 3.62992048e-01],
       [-4.39071238e-01],
       [-5.01997769e-01],
       [-1.64807987e+00],
       [-7.28634417e-01],
       [-1.28700829e+00],
       [-4.91840631e-01],
       [-1.08798969e+00],
       [-1.30176783e+00],
       [-1.14527392e+00],
       [-1.20505393e+00],
       [-4.85135645e-01],
       [-6.38878822e-01],
       [-4.34189230e-01],
       [-1.34591436e+00],
       [-8.13466132e-01],
       [-5.47261417e-01],
       [-1.81899816e-01],
       [-1.61320043e+00],
       [-1.00362813e+00],
       [-1.16978395e+00],
       [-1.39886451e+00],
       [-3.69070083e-01],
       [ 1.38616264e-01],
       [-1.35112369e+00],
       [-5.83317161e-01],
       [-1.17583168e+00],
       [-4.65919375e-01],
       [-8.64838779e-01],
       [-8.65782440e-01],
       [-5.1

In [171]:
scaled_predictions_1d =  scaled_predictions.reshape(-1)
display(scaled_predictions_1d)

array([-1.56514525e-01,  3.78918052e-01, -1.20398891e+00, -1.03173518e+00,
       -1.09153891e+00, -1.16096473e+00, -1.01697111e+00, -1.66051745e+00,
        3.62992048e-01, -4.39071238e-01, -5.01997769e-01, -1.64807987e+00,
       -7.28634417e-01, -1.28700829e+00, -4.91840631e-01, -1.08798969e+00,
       -1.30176783e+00, -1.14527392e+00, -1.20505393e+00, -4.85135645e-01,
       -6.38878822e-01, -4.34189230e-01, -1.34591436e+00, -8.13466132e-01,
       -5.47261417e-01, -1.81899816e-01, -1.61320043e+00, -1.00362813e+00,
       -1.16978395e+00, -1.39886451e+00, -3.69070083e-01,  1.38616264e-01,
       -1.35112369e+00, -5.83317161e-01, -1.17583168e+00, -4.65919375e-01,
       -8.64838779e-01, -8.65782440e-01, -5.17337143e-01, -9.60512698e-01,
       -2.37458557e-01, -6.82788193e-01, -1.14063573e+00, -1.20586610e+00,
       -1.31349421e+00, -7.85214067e-01, -1.05863309e+00, -1.62761056e+00,
       -8.60724032e-01, -6.42384291e-01, -1.14227629e+00, -1.13383746e+00,
       -8.97180438e-01, -

In [172]:
predictions_df = pd.DataFrame({
    'product_id': product_ids,
    'predicted_y': scaled_predictions_1d
})

# Iterar sobre cada fila del DataFrame predictions_df
for index, row in predictions_df.iterrows():
    # Obtener el ID del producto y la predicción escalada para este producto
    product_id = row['product_id']
    scaled_prediction = row['predicted_y']
    
    # Obtener el escalador correspondiente a 'predicted_y' para este producto
    scaler = scalers['y'][(product_id)]
    
    # Aplicar la inversa del escalador a la predicción 'predicted_y' para este producto
    inverse_scaled_prediction = scaler.inverse_transform([[scaled_prediction]])[0][0]
    
    # Reemplazar la predicción escalada con la predicción invertida en el DataFrame final
    predictions_df.at[index, 'predicted_y'] = inverse_scaled_prediction

# Guardar el DataFrame final con las predicciones invertidas
predictions_df.to_csv('predictions.csv', index=False)

print('Todas las predicciones han sido generadas y guardadas en predictions.csv después de aplicar la inversa de los scalers.')


Todas las predicciones han sido generadas y guardadas en predictions.csv después de aplicar la inversa de los scalers.


/var/folders/ls/f33jscqd6sb6bd857yb2j2s00000gp/T/ipykernel_17463/2777572878.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1352.3329032911458' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  predictions_df.at[index, 'predicted_y'] = inverse_scaled_prediction
